# Connectome Analyses

We develop tools to analyze a circuit model's connectome.

In [1]:
import os
import numpy as np
import pandas as pd
from neuro_dmt.library.users.visood.sscx_dissemination.analyses.composition\
    import CompositionAnalysesSuite
from neuro_dmt.models.bluebrain.circuit.atlas import\
    BlueBrainCircuitAtlas
from neuro_dmt.models.bluebrain.circuit.model import\
    BlueBrainCircuitModel,\
    CircuitProvenance
from neuro_dmt.models.bluebrain.circuit.adapter import\
    BlueBrainCircuitAdapter

('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'roi', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'roi', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'roi', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mtype', 'etype', 'synapse_class')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'roi', 'mtype', 'etype', 'synapse_class', 'postsynaptic', 'presynaptic')
('region', 'layer', 'depth', 'height', 'mesocolumn', 'hypercolumn', 'roi')


In [2]:
project =\
    os.path.join(
        "/gpfs/bbp.cscs.ch/project")
proj_sscx_diss=\
    os.path.join(
        project,
        "proj83")
data_sscx_diss=\
    os.path.join(
        proj_sscx_diss,
        "data")
atlases_sscx_diss=\
    os.path.join(
        data_sscx_diss,
        "atlas/S1/MEAN")
circuits_sscx_diss=\
    os.path.join(
        proj_sscx_diss,
        "circuits")
atlas_bio_m =\
    BlueBrainCircuitAtlas(
        path=os.path.join(
                atlases_sscx_diss, "P14-MEAN"))
path_bio_m =\
    os.path.join(
        circuits_sscx_diss,
        "Bio_M/20191206")
circuit_bio_m =\
    BlueBrainCircuitModel(
        path_circuit_data=path_bio_m,
            provenance=CircuitProvenance(
                label="SSCxRatDisseminationBioM",
                authors=["BBP Team"],
                release_date="20191212",
                uri=path_bio_m,
                animal="Wistar Rat",
                age="P14",
                brain_region="SSCx"))
adapter =\
    BlueBrainCircuitAdapter()

In [3]:
from neuro_dmt.library.users.visood.sscx_dissemination.analyses.connectome\
    import PathwaySummary

In [4]:
summary_pathway =\
    PathwaySummary(
        circuit_model=circuit_bio_m,
        adapter=adapter)

In [5]:
cells_eg =\
    adapter.get_cells(
        circuit_bio_m,
        properties={"layer", "mtype", "etype"},
        layer="L2")

In [6]:
cells_eg.head()

,mtype,etype,layer,gid
1311604,L2_TPC:A,cADpyr,2,1311604
1311605,L2_TPC:A,cADpyr,2,1311605
1311606,L2_TPC:A,cADpyr,2,1311606
1311607,L2_TPC:A,cADpyr,2,1311607
1311608,L2_TPC:A,cADpyr,2,1311608


In [7]:
pairs_eg =\
    pd.concat([
        _pairs for _pairs in summary_pathway.get_pairs(
            cells_eg.head(),
            cells_eg.tail())])

ImportError: dlopen(/Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/libsonata.cpython-37m-darwin.so, 2): Library not loaded: /usr/local/lib/libhdf5.101.dylib
  Referenced from: /Users/vishalsoodmuchu/.vmgr_repo/py37/lib/python3.7/site-packages/libsonata.cpython-37m-darwin.so
  Reason: image not found